# Tranformer et modèles pré-entrainés

Tutorial sur les outils huggingface

Nous continuons à travailler sur la classification d'opinion pour ce nouveau TP

In [ ]:
# Importation des bibliothèques nécessaires
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, pipeline
from datasets import load_dataset, Dataset, DatasetDict
import os
import torch

1. Introduction aux composants de base de la librairie HuggingFace

HuggingFace propose deux composants clés :
- AutoTokenizer : pour préparer les données textuelles en un format compatible avec les modèles
- AutoModelForSequenceClassification : pour charger un modèle pré-entrainé pour les tâches de classification

2. Récupération d'un modèle pré-entrainé de classification de sentiment

Utilisation d'un modèle léger pré-entrainé sur des tâches de classification de sentiments

In [ ]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"  # Un modèle léger pour la classification de sentiments

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
# Exemple de phrases à classer
sentences = [
    "I love this product!",
    "This is the worst experience I've ever had.",
    "It's okay, not great but not bad either."
]

In [ ]:
# Tokenisation des phrases
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")

# Prédictions avec le modèle
outputs = model(**inputs)
print(outputs)

In [ ]:
# Utilisation d'une pipeline pour simplifier le processus
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Prédictions directement avec la pipeline
results = classifier(sentences)
for result in results:
    print(result)


## EXERCICE

Faire passer les données des films dans cette chaine de traitement et évaluer les performances

## Fine-tuner ces modèles avec des données 

In [ ]:
from transformers import  Trainer, TrainingArguments
from datasets import load_dataset, Dataset, DatasetDict
import os

# Charger les données locales depuis des répertoires "pos" et "neg"
def load_data_from_dirs(base_dir):
    texts, labels = [], []
    for label, sentiment in enumerate(["neg", "pos"]):
        dir_path = os.path.join(base_dir, sentiment)
        for file_name in os.listdir(dir_path):
            file_path = os.path.join(dir_path, file_name)
            with open(file_path, "r", encoding="utf-8") as f:
                texts.append(f.read().strip())
                labels.append(label)
    return Dataset.from_dict({"text": texts, "label": labels})


In [ ]:

# Charger les ensembles d'entraînement et de test
train_dataset = load_data_from_dirs("data/aclimdb/train")
test_dataset = load_data_from_dirs("data/aclimdb/test")

# Créer un DatasetDict pour HuggingFace Trainer
datasets = DatasetDict({"train": train_dataset, "test": test_dataset})


In [ ]:

# Tokenisation des données
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = datasets.map(tokenize_function, batched=True)


In [ ]:

# Définir les arguments d'entraînement
training_args = TrainingArguments(
    output_dir="./results",          # dossier de sortie
    eval_strategy="epoch",    # stratégie d'évaluation
    learning_rate=2e-5,              # taux d'apprentissage
    per_device_train_batch_size=16,  # taille des batches pour l'entraînement
    per_device_eval_batch_size=16,   # taille des batches pour l'évaluation
    num_train_epochs=1,              # nombre d'époques
    weight_decay=0.01,               # décroissance de poids
    logging_dir="./logs",           # dossier de logs
    logging_steps=10,
)

# Initialisation du Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
)

# Entraînement
trainer.train()


In [ ]:

# Évaluation
trainer.evaluate()

# 4. Utilisation du modèle fine-tuné pour des prédictions
# Exemple de phrases à classer
sentences = [
    "I love this product!",
    "This is the worst experience I've ever had.",
    "It's okay, not great but not bad either."
]

# Utilisation d'une pipeline pour simplifier le processus
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Prédictions directement avec la pipeline
results = classifier(sentences)
for result in results:
    print(result)


# Construction du sujet à partir de la correction

In [ ]:
###  TODO )"," TODO ",\
    txt, flags=re.DOTALL))
f2.close()

### </CORRECTION> ###